# Zipper

In [20]:
data Tree a = Empty | Node a (Tree a) (Tree a) deriving (Show)  

In [21]:
freeTree :: Tree Char  
freeTree =   
    Node 'P'  
    -- 중앙
        -- 좌측 트리
        (Node 'O'  
            (Node 'L'  
                (Node 'N' Empty Empty)  
                (Node 'T' Empty Empty)  
            )  
            (Node 'Y'  
                (Node 'S' Empty Empty)  
                (Node 'A' Empty Empty)  
            )  
        )  
        -- 우측 트리
        (Node 'L'  
            (Node 'W'  
                (Node 'C' Empty Empty)  
                (Node 'R' Empty Empty)  
            )  
            (Node 'A'  
                (Node 'A' Empty Empty)  
                (Node 'C' Empty Empty)  
            )  
        )  

In [4]:
changeToP :: Tree Char -> Tree Char
changeToP (Node x l (Node y (Node _ m n) r)) = Node x l (Node y (Node 'P' m n) r)
-- x: 상위 노드 값
-- l: 좌측 트리
-- y: 우측 트리 노드 값
-- r: 우측 노드의 우측 노드 값
-- m: ...
-- ...

-- 사실상 특정 트리를 바꾸기 위해 작성한 코드

In [5]:
data Direction = L | R deriving (Show)
type Directions = [Direction]

changeToP :: Directions -> Tree Char -> Tree Char  
changeToP (L:ds) (Node x l r) = Node x (changeToP ds l) r  
changeToP (R:ds) (Node x l r) = Node x l (changeToP ds r)  
-- 도착할 때까지 재귀
changeToP [] (Node _ l r) = Node 'P' l r
-- 미리 좌우 지령을 내려서 도착한 다음 수정

## 빵가루 흔적 남기기

In [6]:
data Tree a = Empty | Node a (Tree a) (Tree a) deriving (Show)  
data Direction = L | R deriving (Show)
type BreadScrumbs = [Direction]

In [7]:
goLeft :: (Tree a, BreadScrumbs) -> (Tree a, BreadScrumbs)
goLeft (Node _ l _, bs) = (l, L:bs)

goRight :: (Tree a, BreadScrumbs) -> (Tree a, BreadScrumbs)
goRight (Node _ _ r, bs) = (r, R:bs)

-- 함수를 실행할수롟 breadscrums가 많아진다.

In [ ]:
goLeft (goRight (freeTree, []))
-- 트리의 오른쪽 이동 후 왼쪽 이동

In [9]:
x |> f = f x
-- fsharp |>

In [ ]:
(freeTree, []) |> goRight |> goLeft

## 돌아가기

In [26]:
data Crumb a = LeftCrumb a (Tree a) | RightCrumb a (Tree a) deriving (Show)
-- a == 이전 노드 값
-- 트리 == 가지 못한 방향의 트리

type Breadcrumbs a = [Crumb a]

In [27]:
goLeft :: (Tree a, Breadcrumbs a) -> (Tree a, Breadcrumbs a)
goLeft (Node x l r, bs) = (l, LeftCrumb x r:bs)

goRight :: (Tree a, Breadcrumbs a) -> (Tree a, Breadcrumbs a)
goRight (Node x l r, bs) = (r, RightCrumb x l:bs)

In [28]:
goUp :: (Tree a, Breadcrumbs a) -> (Tree a, Breadcrumbs a)
goUp (t, LeftCrumb x r:bs) = (Node x t r, bs)
goUp (t, RightCrumb x l:bs) = (Node x l t, bs)

In [29]:
type Zipper a = (Tree a, Breadcrumbs a)

### 초점을 둔 트리 조작하기

In [30]:
modify :: (a -> a) -> Zipper a -> Zipper a
modify f (Node x l r, bs) = (Node (f x) l r, bs)
modify f (Empty, bs) = (Empty, bs)
-- 현재 초점을 두고 있는 노드의 값을 수정하기

In [32]:
modify (\_ -> 'P') (goRight (goLeft (freeTree,[])))  

Line 1: Use const
Found:
\ _ -> 'P'
Why not:
const 'P'

(Node 'P' (Node 'S' Empty Empty) (Node 'A' Empty Empty),[RightCrumb 'O' (Node 'L' (Node 'N' Empty Empty) (Node 'T' Empty Empty)),LeftCrumb 'P' (Node 'L' (Node 'W' (Node 'C' Empty Empty) (Node 'R' Empty Empty)) (Node 'A' (Node 'A' Empty Empty) (Node 'C' Empty Empty)))])

In [36]:
(freeTree,[]) |> goLeft |> goRight |> modify (\_ -> 'P')  
-- readablity 향상

Line 1: Use const
Found:
\ _ -> 'P'
Why not:
const 'P'

(Node 'P' (Node 'S' Empty Empty) (Node 'A' Empty Empty),[RightCrumb 'O' (Node 'L' (Node 'N' Empty Empty) (Node 'T' Empty Empty)),LeftCrumb 'P' (Node 'L' (Node 'W' (Node 'C' Empty Empty) (Node 'R' Empty Empty)) (Node 'A' (Node 'A' Empty Empty) (Node 'C' Empty Empty)))])

In [38]:
modify (\_ -> 'X') (goUp newFocus)  
-- newFocus |> goUp |> modify (\_ -> 'X')   동일

Line 1: Use const
Found:
\ _ -> 'X'
Why not:
const 'X'

(Node 'X' (Node 'L' (Node 'N' Empty Empty) (Node 'T' Empty Empty)) (Node 'P' (Node 'S' Empty Empty) (Node 'A' Empty Empty)),[LeftCrumb 'P' (Node 'L' (Node 'W' (Node 'C' Empty Empty) (Node 'R' Empty Empty)) (Node 'A' (Node 'A' Empty Empty) (Node 'C' Empty Empty)))])

In [40]:
attach :: Tree a -> Zipper a -> Zipper a  
attach t (_, bs) = (t, bs)  

In [41]:
farLeft = (freeTree,[]) |> goLeft |> goLeft |> goLeft |> goLeft  
farLeft

(Empty,[LeftCrumb 'N' Empty,LeftCrumb 'L' (Node 'T' Empty Empty),LeftCrumb 'O' (Node 'Y' (Node 'S' Empty Empty) (Node 'A' Empty Empty)),LeftCrumb 'P' (Node 'L' (Node 'W' (Node 'C' Empty Empty) (Node 'R' Empty Empty)) (Node 'A' (Node 'A' Empty Empty) (Node 'C' Empty Empty)))])

In [43]:
newFocus = farLeft |> attach (Node 'Z' Empty Empty)  
newFocus

(Node 'Z' Empty Empty,[LeftCrumb 'N' Empty,LeftCrumb 'L' (Node 'T' Empty Empty),LeftCrumb 'O' (Node 'Y' (Node 'S' Empty Empty) (Node 'A' Empty Empty)),LeftCrumb 'P' (Node 'L' (Node 'W' (Node 'C' Empty Empty) (Node 'R' Empty Empty)) (Node 'A' (Node 'A' Empty Empty) (Node 'C' Empty Empty)))])

In [44]:
topMost :: Zipper a -> Zipper a  
topMost (t,[]) = (t,[])  
-- 이동내용이 없다면 그대로
topMost z = topMost (goUp z)  
-- 이동내용이 있다면 하나 올리고 재귀

## 리스트 지퍼

In [46]:
type ListZipper a = ([a], [a])
-- 이전 요소, 포커스 요소

In [47]:
goForward :: ListZipper a -> ListZipper a
goForward (x:xs, bs) = (xs, x:bs)

goBack :: ListZipper a -> ListZipper a
goBack (xs, b:bs) = (b:xs, bs)

In [50]:
xs = [1,2,3,4]
goForward (xs, [])
goForward ([2,3,4], [1])
goForward ([3,4], [2,1])
goBack ([4], [3,2,1])

([2,3,4],[1])

([3,4],[2,1])

([4],[3,2,1])

([3,4],[2,1])

## 매우 간단한 파일시스템

In [51]:
type Name = String
type Data = String
data FSItem = File Name Data | Folder Name [FSItem] deriving (Show)

In [52]:
myDisk :: FSItem  
myDisk = 
    Folder "root"   
        [ File "goat_yelling_like_man.wmv" "baaaaaa"  
        , File "pope_time.avi" "god bless"  
        , Folder "pics"  
            [ File "ape_throwing_up.jpg" "bleargh"  
            , File "watermelon_smash.gif" "smash!!"  
            , File "skull_man(scary).bmp" "Yikes!"  
            ]  
        , File "dijon_poupon.doc" "best mustard"  
        , Folder "programs"  
            [ File "fartwizard.exe" "10gotofart"  
            , File "owl_bandit.dmg" "mov eax, h00t"  
            , File "not_a_virus.exe" "really not a virus"  
            , Folder "source code"  
                [ File "best_hs_prog.hs" "main = print (fix error)"  
                , File "random.hs" "main = print 4"  
                ]  
            ]  
        ]  

In [53]:
data FSCrumb = FSCrumb Name [FSItem] [FSItem] deriving (Show)
-- 항목 이전에 오는 리스트와, 항목 이후에 오는 리스트
-- 초점을 맞춘 디렉토리에 있는 파일들이 있다.

In [61]:
type FSZipper = (FSItem, [FSCrumb])

In [62]:
fsUp :: FSZipper -> FSZipper
fsUp (item, (FSCrumb name ls rs):bs) = (Folder name (ls ++ [item] ++ rs), bs)

In [63]:
import Data.List (break)  
-- break: 해당하는 순간에 앞과 뒤 리스트를 분리하는 함수
  
fsTo :: Name -> FSZipper -> FSZipper  
fsTo name (Folder folderName items, bs) =   
    let (ls, item:rs) = break (nameIs name) items  

    in  (item, FSCrumb folderName ls rs:bs)  
-- 현재 초점을 두고 있는 폴더에 위치한 파일이나 폴더에 초점을 둫는 함수
  
nameIs :: Name -> FSItem -> Bool  
nameIs name (Folder folderName _) = name == folderName  
nameIs name (File fileName _) = name == fileName  

In [71]:
newFocus = (myDisk,[]) |> fsTo "pics" |> fsTo "skull_man(scary).bmp"  
-- 폴더로 이동 후, 파일로 이동
fst newFocus
snd newFocus



File "skull_man(scary).bmp" "Yikes!"

[FSCrumb "pics" [File "ape_throwing_up.jpg" "bleargh",File "watermelon_smash.gif" "smash!!"] [],FSCrumb "root" [File "goat_yelling_like_man.wmv" "baaaaaa",File "pope_time.avi" "god bless"] [File "dijon_poupon.doc" "best mustard",Folder "programs" [File "fartwizard.exe" "10gotofart",File "owl_bandit.dmg" "mov eax, h00t",File "not_a_virus.exe" "really not a virus",Folder "source code" [File "best_hs_prog.hs" "main = print (fix error)",File "random.hs" "main = print 4"]]]]

In [72]:
newFocus2 = newFocus |> fsUp |> fsTo "watermelon_smash.gif"  

fst newFocus2  
snd newFocus2  

File "watermelon_smash.gif" "smash!!"

[FSCrumb "pics" [File "ape_throwing_up.jpg" "bleargh"] [File "skull_man(scary).bmp" "Yikes!"],FSCrumb "root" [File "goat_yelling_like_man.wmv" "baaaaaa",File "pope_time.avi" "god bless"] [File "dijon_poupon.doc" "best mustard",Folder "programs" [File "fartwizard.exe" "10gotofart",File "owl_bandit.dmg" "mov eax, h00t",File "not_a_virus.exe" "really not a virus",Folder "source code" [File "best_hs_prog.hs" "main = print (fix error)",File "random.hs" "main = print 4"]]]]

### 조작하기

In [74]:
fsRename :: Name -> FSZipper -> FSZipper  
fsRename newName (Folder name items, bs) = (Folder newName items, bs)  
fsRename newName (File name dat, bs) = (File newName dat, bs)  

In [79]:
newFocus = (myDisk,[]) |> fsTo "pics" |> fsRename "cspi" |> fsUp  

fst newFocus
snd newFocus
-- root로 올라와서 crumbs가 비었음

Folder "root" [File "goat_yelling_like_man.wmv" "baaaaaa",File "pope_time.avi" "god bless",Folder "cspi" [File "ape_throwing_up.jpg" "bleargh",File "watermelon_smash.gif" "smash!!",File "skull_man(scary).bmp" "Yikes!"],File "dijon_poupon.doc" "best mustard",Folder "programs" [File "fartwizard.exe" "10gotofart",File "owl_bandit.dmg" "mov eax, h00t",File "not_a_virus.exe" "really not a virus",Folder "source code" [File "best_hs_prog.hs" "main = print (fix error)",File "random.hs" "main = print 4"]]]

[]

In [80]:
fsNewFile :: FSItem -> FSZipper -> FSZipper  
fsNewFile item (Folder folderName items, bs) =   
    (Folder folderName (item:items), bs)  
-- 새로운 항목을 만듬

In [82]:
newFocus = 
    (myDisk,[]) |> 
    fsTo "pics" |> 
    fsNewFile (File "heh.jpg" "lol") |> fsUp  
    
fst newFocus
snd newFocus

Folder "root" [File "goat_yelling_like_man.wmv" "baaaaaa",File "pope_time.avi" "god bless",Folder "pics" [File "heh.jpg" "lol",File "ape_throwing_up.jpg" "bleargh",File "watermelon_smash.gif" "smash!!",File "skull_man(scary).bmp" "Yikes!"],File "dijon_poupon.doc" "best mustard",Folder "programs" [File "fartwizard.exe" "10gotofart",File "owl_bandit.dmg" "mov eax, h00t",File "not_a_virus.exe" "really not a virus",Folder "source code" [File "best_hs_prog.hs" "main = print (fix error)",File "random.hs" "main = print 4"]]]

[]

## Zipper에 실패 컨텍스트 추가하기

In [83]:
goLeft :: Zipper a -> Maybe (Zipper a)  
goLeft (Node x l r, bs) = Just (l, LeftCrumb x r:bs)  
goLeft (Empty, _) = Nothing  
-- 컨텍스트 별 차이를 위해 분리
  
goRight :: Zipper a -> Maybe (Zipper a)  
goRight (Node x l r, bs) = Just (r, RightCrumb x l:bs)  
goRight (Empty, _) = Nothing  
-- 컨텍스트 별 차이를 위해 분리

In [85]:
goLeft (Empty, [])
goLeft (Node 'A' Empty Empty, [])

Nothing

Just (Empty,[LeftCrumb 'A' Empty])

In [87]:
goUp :: Zipper a -> Maybe (Zipper a)  
goUp (t, LeftCrumb x r:bs) = Just (Node x t r, bs)  
goUp (t, RightCrumb x l:bs) = Just (Node x l t, bs)  
goUp (_, []) = Nothing  
-- empty crumbs 위로 올라갈 때 실패 컨텍스트 부여

In [88]:
newFocus = (freeTree,[]) |> goLeft |> goRight  
-- 함수가 모나드를 반환하고, 함수가 모나드를 받지 않으므로 동작하지 않는다.

: 

In [94]:
coolTree = Node 1 Empty (Node 3 Empty Empty)
-- 바인딩하기 위해 return으로 최소 디폴트 콘텍스트를 부여
return (coolTree, []) >>= goRight
return (coolTree, []) >>= goRight >>= goRight
return (coolTree, []) >>= goRight >>= goRight >>= goRight

Line 3: Monad law, left identity
Found:
return (coolTree, []) >>= goRight
Why not:
goRight (coolTree, [])Line 4: Monad law, left identity
Found:
return (coolTree, []) >>= goRight
Why not:
goRight (coolTree, [])Line 5: Monad law, left identity
Found:
return (coolTree, []) >>= goRight
Why not:
goRight (coolTree, [])

Just (Node 3 Empty Empty,[RightCrumb 1 Empty])

Just (Empty,[RightCrumb 3 Empty,RightCrumb 1 Empty])

Nothing